In [1]:
import pandas
import numpy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import statistics


import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

data = pandas.read_csv("finalData.csv", header = [0])
data = pandas.DataFrame.drop(data, columns="date")
data = pandas.DataFrame.drop(data, columns="index")

data.reset_index()
print(data)

cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'sma1', 'sma3',
       'sma7', 'sma14', 'sma21', 'rsi1', 'rsi3', 'rsi7', 'rsi14', 'rsi21',
       'ema1', 'ema3', 'ema7', 'ema14', 'ema21', 'followingClose']
scaler = MinMaxScaler()
scaledData = pandas.DataFrame(scaler.fit_transform(data), columns=cols)
print(scaledData)

data = scaledData.to_numpy()
print(data.shape)

               Open          High           Low         Close     Adj Close  \
0       8108.519043   8123.962891   8077.428711   8104.428711   8104.428711   
1       8105.412109   8119.846680   8075.443359   8110.446777   8110.446777   
2       8111.363770   8129.020996   8081.375488   8087.146484   8087.146484   
3       8090.199707   8094.084473   8072.932617   8079.016602   8079.016602   
4       8079.086426   8114.485840   8074.489746   8112.970703   8112.970703   
...             ...           ...           ...           ...           ...   
16021  54662.910156  54860.796875  54588.640625  54784.527344  54784.527344   
16022  54783.082031  55221.703125  54729.722656  55104.761719  55104.761719   
16023  55034.878906  55220.335938  54768.589844  54804.015625  54804.015625   
16024  54784.734375  55046.371094  54665.535156  54957.445312  54957.445312   
16025  54953.441406  55223.472656  54869.277344  54914.851562  54914.851562   

               sma1          sma3          sma7    

In [2]:
inputData = data[:,:20]
inputData = inputData[:16025,:]
inputData = inputData.reshape(3205,5,20)


outputData = data[:,20:]
outputData = outputData[:16025,]
outputData = outputData.reshape(3205,5,1)
outputData = outputData[:,4:,]
outputData = outputData.reshape(3205,1)

print(inputData)
print(outputData)

[[[0.0567689  0.05383947 0.06168341 ... 0.04191387 0.02908566 0.02549707]
  [0.05672046 0.05377527 0.06165257 ... 0.04199506 0.02913376 0.02553565]
  [0.05681325 0.05391836 0.06174473 ... 0.04195462 0.02911391 0.02552721]
  [0.0564833  0.05337348 0.06161356 ... 0.04197423 0.02912954 0.02554326]
  [0.05631004 0.05369166 0.06163775 ... 0.04216834 0.02924115 0.02562603]]

 [[0.05687287 0.05375401 0.06124697 ... 0.04204573 0.02917786 0.02558715]
  [0.05617078 0.05340688 0.06124764 ... 0.04214542 0.02923756 0.02563471]
  [0.05636789 0.05332072 0.06119435 ... 0.04210994 0.02922118 0.02562902]
  [0.05606258 0.05324702 0.06146258 ... 0.0422674  0.0293119  0.02569692]
  [0.05639625 0.05346939 0.06156677 ... 0.04225709 0.02930932 0.02570011]]

 [[0.05615669 0.05330128 0.06128569 ... 0.04239313 0.0293872  0.0257596 ]
  [0.05650316 0.0539756  0.0617486  ... 0.04260913 0.02950864 0.02584981]
  [0.05711613 0.05443877 0.06247159 ... 0.04270486 0.02956463 0.02589418]
  [0.05725033 0.05422264 0.0624017

In [3]:
xTrain, xTest, yTrain, yTest = train_test_split(inputData, outputData, test_size=0.1, random_state=42)
print(xTrain.shape)
print(xTest.shape)

(2884, 5, 20)
(321, 5, 20)


In [4]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(5, return_sequences=True, input_shape=(5,20)))
#model.add(LSTM(5, return_sequences=True, input_shape=(5,20)))
model.add(LSTM(5, activation="relu"))
model.add(Dense(1))
model.compile(loss='mean_absolute_percentage_error', optimizer='adam')
model.fit(xTrain, yTrain, epochs=500, batch_size=1, verbose=2,validation_data=(xTest,yTest))

Epoch 1/500
2884/2884 - 10s - loss: 10.0032 - val_loss: 4.4926 - 10s/epoch - 3ms/step
Epoch 2/500
2884/2884 - 8s - loss: 4.4763 - val_loss: 5.3862 - 8s/epoch - 3ms/step
Epoch 3/500
2884/2884 - 8s - loss: 3.8987 - val_loss: 3.1758 - 8s/epoch - 3ms/step
Epoch 4/500
2884/2884 - 8s - loss: 3.6642 - val_loss: 5.3344 - 8s/epoch - 3ms/step
Epoch 5/500
2884/2884 - 8s - loss: 3.6145 - val_loss: 3.9729 - 8s/epoch - 3ms/step
Epoch 6/500
2884/2884 - 7s - loss: 3.7444 - val_loss: 2.6459 - 7s/epoch - 3ms/step
Epoch 7/500
2884/2884 - 7s - loss: 3.2993 - val_loss: 5.8674 - 7s/epoch - 3ms/step
Epoch 8/500
2884/2884 - 7s - loss: 3.3638 - val_loss: 3.8771 - 7s/epoch - 3ms/step
Epoch 9/500
2884/2884 - 7s - loss: 3.2892 - val_loss: 3.8396 - 7s/epoch - 3ms/step
Epoch 10/500
2884/2884 - 7s - loss: 3.2362 - val_loss: 3.0905 - 7s/epoch - 3ms/step
Epoch 11/500
2884/2884 - 7s - loss: 3.0016 - val_loss: 3.9371 - 7s/epoch - 3ms/step
Epoch 12/500
2884/2884 - 7s - loss: 2.9472 - val_loss: 2.6508 - 7s/epoch - 3ms/ste

Epoch 98/500
2884/2884 - 8s - loss: 1.9615 - val_loss: 1.2586 - 8s/epoch - 3ms/step
Epoch 99/500
2884/2884 - 8s - loss: 2.0775 - val_loss: 1.6734 - 8s/epoch - 3ms/step
Epoch 100/500
2884/2884 - 8s - loss: 2.0094 - val_loss: 1.6533 - 8s/epoch - 3ms/step
Epoch 101/500
2884/2884 - 8s - loss: 2.1221 - val_loss: 2.5987 - 8s/epoch - 3ms/step
Epoch 102/500
2884/2884 - 8s - loss: 1.9974 - val_loss: 1.5337 - 8s/epoch - 3ms/step
Epoch 103/500
2884/2884 - 9s - loss: 2.0148 - val_loss: 1.6568 - 9s/epoch - 3ms/step
Epoch 104/500
2884/2884 - 8s - loss: 2.0079 - val_loss: 1.1723 - 8s/epoch - 3ms/step
Epoch 105/500
2884/2884 - 8s - loss: 2.0848 - val_loss: 4.1982 - 8s/epoch - 3ms/step
Epoch 106/500
2884/2884 - 8s - loss: 1.9724 - val_loss: 1.8118 - 8s/epoch - 3ms/step
Epoch 107/500
2884/2884 - 8s - loss: 1.9434 - val_loss: 2.4626 - 8s/epoch - 3ms/step
Epoch 108/500
2884/2884 - 8s - loss: 2.0694 - val_loss: 1.2778 - 8s/epoch - 3ms/step
Epoch 109/500
2884/2884 - 8s - loss: 1.9124 - val_loss: 1.3585 - 8s

Epoch 195/500
2884/2884 - 8s - loss: 1.8788 - val_loss: 1.1784 - 8s/epoch - 3ms/step
Epoch 196/500
2884/2884 - 8s - loss: 1.6551 - val_loss: 1.3698 - 8s/epoch - 3ms/step
Epoch 197/500
2884/2884 - 8s - loss: 1.8250 - val_loss: 1.1639 - 8s/epoch - 3ms/step
Epoch 198/500
2884/2884 - 8s - loss: 1.7412 - val_loss: 1.1521 - 8s/epoch - 3ms/step
Epoch 199/500
2884/2884 - 8s - loss: 1.8373 - val_loss: 1.4873 - 8s/epoch - 3ms/step
Epoch 200/500
2884/2884 - 8s - loss: 1.7577 - val_loss: 2.8961 - 8s/epoch - 3ms/step
Epoch 201/500
2884/2884 - 8s - loss: 1.5913 - val_loss: 1.2021 - 8s/epoch - 3ms/step
Epoch 202/500
2884/2884 - 8s - loss: 1.8335 - val_loss: 2.1250 - 8s/epoch - 3ms/step
Epoch 203/500
2884/2884 - 8s - loss: 1.7671 - val_loss: 1.0873 - 8s/epoch - 3ms/step
Epoch 204/500
2884/2884 - 8s - loss: 1.7626 - val_loss: 1.1421 - 8s/epoch - 3ms/step
Epoch 205/500
2884/2884 - 8s - loss: 1.6935 - val_loss: 1.2201 - 8s/epoch - 3ms/step
Epoch 206/500
2884/2884 - 8s - loss: 1.7935 - val_loss: 1.9564 - 

Epoch 292/500
2884/2884 - 8s - loss: 1.6139 - val_loss: 0.9970 - 8s/epoch - 3ms/step
Epoch 293/500
2884/2884 - 8s - loss: 1.6814 - val_loss: 1.0551 - 8s/epoch - 3ms/step
Epoch 294/500
2884/2884 - 8s - loss: 1.6732 - val_loss: 1.7244 - 8s/epoch - 3ms/step
Epoch 295/500
2884/2884 - 8s - loss: 1.6708 - val_loss: 1.9817 - 8s/epoch - 3ms/step
Epoch 296/500
2884/2884 - 8s - loss: 1.7465 - val_loss: 3.1334 - 8s/epoch - 3ms/step
Epoch 297/500
2884/2884 - 8s - loss: 1.6311 - val_loss: 0.9042 - 8s/epoch - 3ms/step
Epoch 298/500
2884/2884 - 8s - loss: 1.7357 - val_loss: 2.5191 - 8s/epoch - 3ms/step
Epoch 299/500
2884/2884 - 8s - loss: 1.6818 - val_loss: 2.4405 - 8s/epoch - 3ms/step
Epoch 300/500
2884/2884 - 8s - loss: 1.5587 - val_loss: 4.4865 - 8s/epoch - 3ms/step
Epoch 301/500
2884/2884 - 8s - loss: 1.8164 - val_loss: 2.1036 - 8s/epoch - 3ms/step
Epoch 302/500
2884/2884 - 8s - loss: 1.8214 - val_loss: 2.5928 - 8s/epoch - 3ms/step
Epoch 303/500
2884/2884 - 8s - loss: 1.6992 - val_loss: 1.0934 - 

Epoch 389/500
2884/2884 - 8s - loss: 1.5777 - val_loss: 1.7434 - 8s/epoch - 3ms/step
Epoch 390/500
2884/2884 - 8s - loss: 1.4606 - val_loss: 0.9622 - 8s/epoch - 3ms/step
Epoch 391/500
2884/2884 - 8s - loss: 1.4982 - val_loss: 0.9492 - 8s/epoch - 3ms/step
Epoch 392/500
2884/2884 - 8s - loss: 1.7187 - val_loss: 2.8068 - 8s/epoch - 3ms/step
Epoch 393/500
2884/2884 - 8s - loss: 1.6033 - val_loss: 1.1192 - 8s/epoch - 3ms/step
Epoch 394/500
2884/2884 - 8s - loss: 1.6593 - val_loss: 5.8689 - 8s/epoch - 3ms/step
Epoch 395/500
2884/2884 - 8s - loss: 1.6971 - val_loss: 0.9933 - 8s/epoch - 3ms/step
Epoch 396/500
2884/2884 - 8s - loss: 1.5476 - val_loss: 1.6853 - 8s/epoch - 3ms/step
Epoch 397/500
2884/2884 - 8s - loss: 1.6643 - val_loss: 0.9943 - 8s/epoch - 3ms/step
Epoch 398/500
2884/2884 - 8s - loss: 1.6178 - val_loss: 1.1923 - 8s/epoch - 3ms/step
Epoch 399/500
2884/2884 - 8s - loss: 1.6755 - val_loss: 1.1575 - 8s/epoch - 3ms/step
Epoch 400/500
2884/2884 - 8s - loss: 1.6498 - val_loss: 2.2672 - 

Epoch 486/500
2884/2884 - 11s - loss: 1.5653 - val_loss: 1.1423 - 11s/epoch - 4ms/step
Epoch 487/500
2884/2884 - 9s - loss: 1.6573 - val_loss: 2.0582 - 9s/epoch - 3ms/step
Epoch 488/500
2884/2884 - 9s - loss: 1.5904 - val_loss: 1.5023 - 9s/epoch - 3ms/step
Epoch 489/500
2884/2884 - 9s - loss: 1.6776 - val_loss: 2.1638 - 9s/epoch - 3ms/step
Epoch 490/500
2884/2884 - 8s - loss: 1.5778 - val_loss: 1.3227 - 8s/epoch - 3ms/step
Epoch 491/500
2884/2884 - 10s - loss: 1.5196 - val_loss: 2.4872 - 10s/epoch - 3ms/step
Epoch 492/500
2884/2884 - 8s - loss: 1.5358 - val_loss: 1.3623 - 8s/epoch - 3ms/step
Epoch 493/500
2884/2884 - 8s - loss: 1.5991 - val_loss: 2.4027 - 8s/epoch - 3ms/step
Epoch 494/500
2884/2884 - 8s - loss: 1.6144 - val_loss: 1.5603 - 8s/epoch - 3ms/step
Epoch 495/500
2884/2884 - 8s - loss: 1.6065 - val_loss: 1.0811 - 8s/epoch - 3ms/step
Epoch 496/500
2884/2884 - 8s - loss: 1.5085 - val_loss: 6.4537 - 8s/epoch - 3ms/step
Epoch 497/500
2884/2884 - 8s - loss: 1.6554 - val_loss: 1.241

In [23]:
inputData6 = data[:,:20]
inputData6 = inputData6[:16020,:]
inputData6 = inputData6.reshape(-1,10,20)
print(inputData6.shape)

outputData6 = data[:,20:]
outputData6 = outputData6[:16020,]
outputData6 = outputData6.reshape(-1,10,1)
outputData6 = outputData6[:,9:,]
outputData6 = outputData6.reshape(-1,1)
print(outputData6.shape)

xTrain6, xTest6, yTrain6, yTest6 = train_test_split(inputData6, outputData6, test_size=0.1, random_state=42)

(1602, 10, 20)
(1602, 1)


In [30]:
# create and fit the LSTM network
model6 = Sequential()
model6.add(LSTM(5, return_sequences=True, input_shape=(10,20)))
#model.add(LSTM(5, return_sequences=True, input_shape=(5,20)))
model6.add(LSTM(5, activation="relu"))
model6.add(Dense(1))
model6.compile(loss='mean_absolute_percentage_error', optimizer='adam')
model6.fit(xTrain6, yTrain6, epochs=16000, batch_size=32, verbose=2,validation_data=(xTest6,yTest6))

Epoch 1/16000
46/46 - 3s - loss: 173.8366 - val_loss: 71.6851 - 3s/epoch - 63ms/step
Epoch 2/16000
46/46 - 0s - loss: 71.0767 - val_loss: 61.0860 - 338ms/epoch - 7ms/step
Epoch 3/16000
46/46 - 0s - loss: 66.6353 - val_loss: 58.9526 - 331ms/epoch - 7ms/step
Epoch 4/16000
46/46 - 0s - loss: 65.2508 - val_loss: 58.7035 - 333ms/epoch - 7ms/step
Epoch 5/16000
46/46 - 0s - loss: 63.7786 - val_loss: 57.0283 - 331ms/epoch - 7ms/step
Epoch 6/16000
46/46 - 0s - loss: 61.5580 - val_loss: 54.4530 - 330ms/epoch - 7ms/step
Epoch 7/16000
46/46 - 0s - loss: 58.8272 - val_loss: 51.6991 - 341ms/epoch - 7ms/step
Epoch 8/16000
46/46 - 0s - loss: 56.4927 - val_loss: 49.7403 - 329ms/epoch - 7ms/step
Epoch 9/16000
46/46 - 0s - loss: 53.2317 - val_loss: 47.0173 - 327ms/epoch - 7ms/step
Epoch 10/16000
46/46 - 0s - loss: 54.1141 - val_loss: 45.1777 - 329ms/epoch - 7ms/step
Epoch 11/16000
46/46 - 0s - loss: 49.5477 - val_loss: 43.9618 - 331ms/epoch - 7ms/step
Epoch 12/16000
46/46 - 0s - loss: 48.7136 - val_loss:

Epoch 96/16000
46/46 - 0s - loss: 8.8701 - val_loss: 9.1947 - 321ms/epoch - 7ms/step
Epoch 97/16000
46/46 - 0s - loss: 8.6243 - val_loss: 8.0361 - 327ms/epoch - 7ms/step
Epoch 98/16000
46/46 - 0s - loss: 8.5814 - val_loss: 7.9754 - 342ms/epoch - 7ms/step
Epoch 99/16000
46/46 - 0s - loss: 8.4753 - val_loss: 9.7664 - 333ms/epoch - 7ms/step
Epoch 100/16000
46/46 - 0s - loss: 8.5192 - val_loss: 8.8172 - 327ms/epoch - 7ms/step
Epoch 101/16000
46/46 - 0s - loss: 8.0754 - val_loss: 8.0210 - 475ms/epoch - 10ms/step
Epoch 102/16000
46/46 - 0s - loss: 8.2331 - val_loss: 8.6184 - 364ms/epoch - 8ms/step
Epoch 103/16000
46/46 - 0s - loss: 8.1974 - val_loss: 10.7608 - 327ms/epoch - 7ms/step
Epoch 104/16000
46/46 - 0s - loss: 8.7248 - val_loss: 9.5729 - 368ms/epoch - 8ms/step
Epoch 105/16000
46/46 - 0s - loss: 9.2609 - val_loss: 9.3208 - 431ms/epoch - 9ms/step
Epoch 106/16000
46/46 - 0s - loss: 8.8687 - val_loss: 7.4428 - 375ms/epoch - 8ms/step
Epoch 107/16000
46/46 - 0s - loss: 7.6077 - val_loss: 10

Epoch 192/16000
46/46 - 0s - loss: 3.3896 - val_loss: 5.3602 - 322ms/epoch - 7ms/step
Epoch 193/16000
46/46 - 0s - loss: 3.5759 - val_loss: 4.4073 - 320ms/epoch - 7ms/step
Epoch 194/16000
46/46 - 0s - loss: 3.4771 - val_loss: 3.7801 - 320ms/epoch - 7ms/step
Epoch 195/16000
46/46 - 0s - loss: 3.0741 - val_loss: 8.9878 - 320ms/epoch - 7ms/step
Epoch 196/16000
46/46 - 0s - loss: 6.9230 - val_loss: 4.2658 - 320ms/epoch - 7ms/step
Epoch 197/16000
46/46 - 0s - loss: 3.3043 - val_loss: 6.2087 - 320ms/epoch - 7ms/step
Epoch 198/16000
46/46 - 0s - loss: 3.3443 - val_loss: 2.9620 - 319ms/epoch - 7ms/step
Epoch 199/16000
46/46 - 0s - loss: 3.0956 - val_loss: 4.6862 - 323ms/epoch - 7ms/step
Epoch 200/16000
46/46 - 0s - loss: 3.5213 - val_loss: 4.0450 - 320ms/epoch - 7ms/step
Epoch 201/16000
46/46 - 0s - loss: 3.3661 - val_loss: 2.9903 - 320ms/epoch - 7ms/step
Epoch 202/16000
46/46 - 0s - loss: 3.2791 - val_loss: 2.6551 - 327ms/epoch - 7ms/step
Epoch 203/16000
46/46 - 0s - loss: 3.1443 - val_loss: 

Epoch 288/16000
46/46 - 0s - loss: 2.2437 - val_loss: 2.6416 - 321ms/epoch - 7ms/step
Epoch 289/16000
46/46 - 0s - loss: 2.4426 - val_loss: 3.4899 - 318ms/epoch - 7ms/step
Epoch 290/16000
46/46 - 0s - loss: 2.5574 - val_loss: 1.9382 - 318ms/epoch - 7ms/step
Epoch 291/16000
46/46 - 0s - loss: 2.1988 - val_loss: 1.8894 - 322ms/epoch - 7ms/step
Epoch 292/16000
46/46 - 0s - loss: 2.3704 - val_loss: 1.8263 - 323ms/epoch - 7ms/step
Epoch 293/16000
46/46 - 0s - loss: 2.0505 - val_loss: 3.0204 - 320ms/epoch - 7ms/step
Epoch 294/16000
46/46 - 0s - loss: 2.4356 - val_loss: 1.7288 - 319ms/epoch - 7ms/step
Epoch 295/16000
46/46 - 0s - loss: 2.2265 - val_loss: 6.0176 - 319ms/epoch - 7ms/step
Epoch 296/16000
46/46 - 0s - loss: 2.7325 - val_loss: 3.0600 - 320ms/epoch - 7ms/step
Epoch 297/16000
46/46 - 0s - loss: 3.1643 - val_loss: 2.1294 - 318ms/epoch - 7ms/step
Epoch 298/16000
46/46 - 0s - loss: 2.3734 - val_loss: 5.2654 - 320ms/epoch - 7ms/step
Epoch 299/16000
46/46 - 0s - loss: 3.4634 - val_loss: 

Epoch 384/16000
46/46 - 0s - loss: 2.6066 - val_loss: 2.3771 - 322ms/epoch - 7ms/step
Epoch 385/16000
46/46 - 0s - loss: 2.0828 - val_loss: 1.9527 - 317ms/epoch - 7ms/step
Epoch 386/16000
46/46 - 0s - loss: 1.9746 - val_loss: 2.3205 - 319ms/epoch - 7ms/step
Epoch 387/16000
46/46 - 0s - loss: 2.1460 - val_loss: 1.8065 - 320ms/epoch - 7ms/step
Epoch 388/16000
46/46 - 0s - loss: 2.4162 - val_loss: 3.4686 - 316ms/epoch - 7ms/step
Epoch 389/16000
46/46 - 0s - loss: 1.9771 - val_loss: 1.5723 - 319ms/epoch - 7ms/step
Epoch 390/16000
46/46 - 0s - loss: 1.7934 - val_loss: 3.1655 - 318ms/epoch - 7ms/step
Epoch 391/16000
46/46 - 0s - loss: 2.3670 - val_loss: 2.6433 - 318ms/epoch - 7ms/step
Epoch 392/16000
46/46 - 0s - loss: 2.0207 - val_loss: 3.6090 - 320ms/epoch - 7ms/step
Epoch 393/16000
46/46 - 0s - loss: 2.3507 - val_loss: 1.9561 - 319ms/epoch - 7ms/step
Epoch 394/16000
46/46 - 0s - loss: 1.8354 - val_loss: 2.2283 - 319ms/epoch - 7ms/step
Epoch 395/16000
46/46 - 0s - loss: 1.9586 - val_loss: 

...

46/46 - 0s - loss: 0.7580 - val_loss: 0.8869 - 338ms/epoch - 7ms/step
Epoch 15623/16000
46/46 - 0s - loss: 0.8158 - val_loss: 0.7891 - 363ms/epoch - 8ms/step
Epoch 15624/16000
46/46 - 0s - loss: 0.7752 - val_loss: 0.9870 - 357ms/epoch - 8ms/step
Epoch 15625/16000
46/46 - 0s - loss: 0.8326 - val_loss: 1.4207 - 366ms/epoch - 8ms/step
Epoch 15626/16000
46/46 - 0s - loss: 0.9721 - val_loss: 0.8068 - 343ms/epoch - 7ms/step
Epoch 15627/16000
46/46 - 0s - loss: 0.8091 - val_loss: 0.9614 - 330ms/epoch - 7ms/step
Epoch 15628/16000
46/46 - 0s - loss: 0.7840 - val_loss: 0.7487 - 358ms/epoch - 8ms/step
Epoch 15629/16000
46/46 - 0s - loss: 0.8169 - val_loss: 0.8151 - 331ms/epoch - 7ms/step
Epoch 15630/16000
46/46 - 0s - loss: 0.7141 - val_loss: 1.9044 - 336ms/epoch - 7ms/step
Epoch 15631/16000
46/46 - 0s - loss: 1.8018 - val_loss: 1.7039 - 332ms/epoch - 7ms/step
Epoch 15632/16000
46/46 - 0s - loss: 0.9911 - val_loss: 1.0778 - 330ms/epoch - 7ms/step
Epoch 15633/16000
46/46 - 0s - loss: 0.9356 - val_

Epoch 15716/16000
46/46 - 0s - loss: 0.8926 - val_loss: 1.2375 - 391ms/epoch - 9ms/step
Epoch 15717/16000
46/46 - 0s - loss: 0.7914 - val_loss: 0.8332 - 381ms/epoch - 8ms/step
Epoch 15718/16000
46/46 - 0s - loss: 0.7203 - val_loss: 0.8542 - 382ms/epoch - 8ms/step
Epoch 15719/16000
46/46 - 0s - loss: 0.7782 - val_loss: 0.7679 - 399ms/epoch - 9ms/step
Epoch 15720/16000
46/46 - 0s - loss: 0.7563 - val_loss: 1.0618 - 411ms/epoch - 9ms/step
Epoch 15721/16000
46/46 - 0s - loss: 0.8217 - val_loss: 0.9387 - 379ms/epoch - 8ms/step
Epoch 15722/16000
46/46 - 0s - loss: 0.7714 - val_loss: 0.7732 - 391ms/epoch - 9ms/step
Epoch 15723/16000
46/46 - 0s - loss: 0.8929 - val_loss: 0.9015 - 381ms/epoch - 8ms/step
Epoch 15724/16000
46/46 - 0s - loss: 0.8138 - val_loss: 1.0552 - 384ms/epoch - 8ms/step
Epoch 15725/16000
46/46 - 0s - loss: 0.8141 - val_loss: 0.9826 - 381ms/epoch - 8ms/step
Epoch 15726/16000
46/46 - 0s - loss: 0.8170 - val_loss: 1.3349 - 387ms/epoch - 8ms/step
Epoch 15727/16000
46/46 - 0s - l

46/46 - 0s - loss: 0.7530 - val_loss: 0.7603 - 388ms/epoch - 8ms/step
Epoch 15810/16000
46/46 - 0s - loss: 0.9072 - val_loss: 1.2718 - 381ms/epoch - 8ms/step
Epoch 15811/16000
46/46 - 0s - loss: 0.7270 - val_loss: 0.7237 - 383ms/epoch - 8ms/step
Epoch 15812/16000
46/46 - 0s - loss: 0.8437 - val_loss: 1.4907 - 382ms/epoch - 8ms/step
Epoch 15813/16000
46/46 - 0s - loss: 0.9020 - val_loss: 1.4600 - 406ms/epoch - 9ms/step
Epoch 15814/16000
46/46 - 0s - loss: 0.9103 - val_loss: 0.9041 - 388ms/epoch - 8ms/step
Epoch 15815/16000
46/46 - 0s - loss: 0.7762 - val_loss: 0.7695 - 384ms/epoch - 8ms/step
Epoch 15816/16000
46/46 - 0s - loss: 0.7660 - val_loss: 0.9385 - 386ms/epoch - 8ms/step
Epoch 15817/16000
46/46 - 0s - loss: 1.0128 - val_loss: 0.7741 - 378ms/epoch - 8ms/step
Epoch 15818/16000
46/46 - 0s - loss: 0.7292 - val_loss: 0.7952 - 376ms/epoch - 8ms/step
Epoch 15819/16000
46/46 - 0s - loss: 0.8491 - val_loss: 0.8315 - 377ms/epoch - 8ms/step
Epoch 15820/16000
46/46 - 0s - loss: 0.7828 - val_

Epoch 15903/16000
46/46 - 0s - loss: 0.8832 - val_loss: 0.7629 - 382ms/epoch - 8ms/step
Epoch 15904/16000
46/46 - 0s - loss: 0.8481 - val_loss: 0.7511 - 384ms/epoch - 8ms/step
Epoch 15905/16000
46/46 - 0s - loss: 0.7468 - val_loss: 0.9080 - 417ms/epoch - 9ms/step
Epoch 15906/16000
46/46 - 0s - loss: 1.0795 - val_loss: 1.8614 - 377ms/epoch - 8ms/step
Epoch 15907/16000
46/46 - 0s - loss: 1.0489 - val_loss: 0.7165 - 380ms/epoch - 8ms/step
Epoch 15908/16000
46/46 - 0s - loss: 0.7976 - val_loss: 1.2084 - 384ms/epoch - 8ms/step
Epoch 15909/16000
46/46 - 0s - loss: 0.7899 - val_loss: 1.1111 - 391ms/epoch - 9ms/step
Epoch 15910/16000
46/46 - 0s - loss: 0.8512 - val_loss: 0.7689 - 381ms/epoch - 8ms/step
Epoch 15911/16000
46/46 - 0s - loss: 0.7760 - val_loss: 0.8471 - 383ms/epoch - 8ms/step
Epoch 15912/16000
46/46 - 0s - loss: 0.7642 - val_loss: 1.4119 - 390ms/epoch - 8ms/step
Epoch 15913/16000
46/46 - 0s - loss: 0.8482 - val_loss: 1.1816 - 382ms/epoch - 8ms/step
Epoch 15914/16000
46/46 - 0s - l

46/46 - 0s - loss: 0.8967 - val_loss: 0.7186 - 381ms/epoch - 8ms/step
Epoch 15997/16000
46/46 - 0s - loss: 0.8415 - val_loss: 1.2673 - 393ms/epoch - 9ms/step
Epoch 15998/16000
46/46 - 0s - loss: 1.0042 - val_loss: 1.0424 - 383ms/epoch - 8ms/step
Epoch 15999/16000
46/46 - 0s - loss: 0.7377 - val_loss: 0.7463 - 387ms/epoch - 8ms/step
Epoch 16000/16000
46/46 - 0s - loss: 0.7681 - val_loss: 1.1072 - 394ms/epoch - 9ms/step


In [27]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(5, return_sequences=True, input_shape=(5,20)))
#model.add(LSTM(5, return_sequences=True, input_shape=(5,20)))
model.add(LSTM(5, activation="relu"))
model.add(Dense(1))
model.compile(loss='mean_absolute_percentage_error', optimizer='adam')
model.fit(xTrain, yTrain, epochs=100, batch_size=1, verbose=2,validation_data=(xTest,yTest))

Epoch 1/100
2884/2884 - 10s - loss: 11.6844 - val_loss: 7.2414 - 10s/epoch - 3ms/step
Epoch 2/100
2884/2884 - 8s - loss: 5.3931 - val_loss: 4.7629 - 8s/epoch - 3ms/step
Epoch 3/100
2884/2884 - 8s - loss: 4.9180 - val_loss: 5.5822 - 8s/epoch - 3ms/step
Epoch 4/100
2884/2884 - 8s - loss: 4.4587 - val_loss: 2.5783 - 8s/epoch - 3ms/step
Epoch 5/100
2884/2884 - 8s - loss: 3.9801 - val_loss: 3.8450 - 8s/epoch - 3ms/step
Epoch 6/100
2884/2884 - 8s - loss: 3.6787 - val_loss: 38.1010 - 8s/epoch - 3ms/step
Epoch 7/100
2884/2884 - 8s - loss: 3.6527 - val_loss: 3.6541 - 8s/epoch - 3ms/step
Epoch 8/100
2884/2884 - 8s - loss: 3.4420 - val_loss: 7.7201 - 8s/epoch - 3ms/step
Epoch 9/100
2884/2884 - 8s - loss: 3.3500 - val_loss: 6.7555 - 8s/epoch - 3ms/step
Epoch 10/100
2884/2884 - 8s - loss: 3.5663 - val_loss: 11.7315 - 8s/epoch - 3ms/step
Epoch 11/100
2884/2884 - 8s - loss: 3.2671 - val_loss: 3.2539 - 8s/epoch - 3ms/step
Epoch 12/100
2884/2884 - 8s - loss: 3.2650 - val_loss: 3.6100 - 8s/epoch - 3ms/s

Epoch 99/100
2884/2884 - 8s - loss: 1.9762 - val_loss: 1.7724 - 8s/epoch - 3ms/step
Epoch 100/100
2884/2884 - 8s - loss: 2.0293 - val_loss: 1.4314 - 8s/epoch - 3ms/step


In [31]:
testPredict = model6.predict(xTest6)
print(len(testPredict), len(yTest6))
diffArray = []
diffSum = 0

for i in range(0, len(yTest6)):
    diff = abs(yTest6[i]-testPredict[i])
    percent = (diff/yTest6[i])*100
    diffArray.append(percent)
    diffSum = diffSum + percent

avgPE = sum(diffArray)
avgPE = avgPE/len(diffArray)

testPredictSum = numpy.sum(testPredict)
yTestSum = numpy.sum(yTest6)

avgPE = (yTestSum-testPredictSum)/len(diffArray)
    
print(avgPE)
model6.save("model10Step")


161 161
-0.000971211542214557


INFO:tensorflow:Assets written to: model10Step/assets


INFO:tensorflow:Assets written to: model10Step/assets
